In [1]:
import tensorflow as tf
import numpy as np
import cv2

/home/kiizuka/.venv/python3-tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kiizuka/.venv/python3-tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kiizuka/.venv/python3-tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kiizuka/.venv/python3-tf/lib/python3.6/site-packages

In [12]:
img_path = './italia.jpg'
img = cv2.imread(img_path)
print(img.shape)
img = cv2.resize(img, (80, 53))
print(img.shape)
# print(img)
# for i in range(img.shape[0]):
#     for j in range(img.shape[1]):
#         for k in range(img.shape[2]):
#             print(img[i][j])
# print(img[10][39][1])
trans_img = img.transpose([2, 0, 1])
print(trans_img.shape)
# print(trans_img[1][10][39])
for i in range(trans_img.shape[0]):
    for j in range(trans_img.shape[1]):
        for k in range(trans_img.shape[2]):
            if(trans_img[i][j][k] != img[j][k][i]):
                print("The wrong transpose data")
                print("CH::{}, H::{}, W::{}".format(i, j, k))

(534, 800, 3)
(53, 80, 3)
(3, 53, 80)
